In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/mount/studenten-temp1/users/godberja/SpokenDialogSystems/college_adviser") # TODO use own path 

from soloist.examples.college_bot.soloist.server import *
from soloist.examples.college_bot.collegebot_server import *

In [3]:
import json
from nltk.translate.bleu_score import corpus_bleu

In [4]:
# load the test dialogues
with open("/mount/studenten-temp1/users/godberja/SpokenDialogSystems/college_adviser/evaluation/test_dialogues.json","r",encoding="utf8") as infile:
    test_data = json.load(infile)

In [5]:
args.model_name_or_path = '/mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/college_model'
main()

Some weights of the model checkpoint at /mount/studenten-temp1/users/zabereus/adviser/soloist_env/soloist/examples/college_bot/college_model were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
response_ex, beliefstate_ex = predictor(["i'm looking for a college in new haven"])
print(response_ex)
print(beliefstate_ex)
print()
response_ex, beliefstate_ex = predictor(["i want to study architecture"])
print(response_ex)
print(beliefstate_ex)
print()
response_ex, beliefstate_ex = predictor(["i'm looking for a college in new haven","which subject would you like to study there ?","i want to study architecture"])
print(response_ex)
print(beliefstate_ex)

sampled results [' state = new haven, options = 7system : what subject would you like to study?']
response what subject would you like to study ?, bs {'state': 'new haven', 'options': '7'}
what subject would you like to study ?
{'state': 'new haven', 'options': '7'}

sampled results [' state = california ; area = architecture, options = 5system : how expensive would you like the college to be?']
response how expensive would you like the college to be ?, bs {'state': 'california', 'area': 'architecture', 'options': '5'}
how expensive would you like the college to be ?
{'state': 'california', 'area': 'architecture', 'options': '5'}

sampled results [' state = new haven, area = architecture, options = 7system : how expensive would you like the college to be?']
response how expensive would you like the college to be ?, bs {'state': 'new haven', 'area': 'architecture', 'options': '7'}
how expensive would you like the college to be ?
{'state': 'new haven', 'area': 'architecture', 'options': 

In [15]:
# make the predictions and save them to file
predictions = []
for dialogue in test_data:
    prediction = dict()
    prediction = copy.deepcopy(dialogue)
    reply, belief = predictor(prediction["history"])
    prediction["predicted_belief"] = belief
    prediction["predicted_reply"] = reply
    predictions += [prediction]
# with open("predictions.json","w",encoding="utf8") as outfile:
#     json.dump(predictions, outfile,indent=4)

sampled results [' state = virginia techsystem : [name] is a [control] top university in [city], [state] with [students] undergraduat']
response [name] is a [control] top university in [city] , [state] with [students] undergraduat, bs {'state': 'virginia tech'}
sampled results [' control = public; region = southwest ; area = linguistics ; name = virginia techsystem : [name] is often called [alias].']
response [name] is often called [alias] ., bs {'control': 'public', 'region': 'southwest', 'area': 'linguistics', 'name': 'virginia tech'}
sampled results [' area = artssystem : do you have any other requirements for the college?']
response do you have any other requirements for the college ?, bs {'area': 'arts'}
sampled results [" control = public; region = southwest ; area = computer science ; name = university of virginia-main campussystem : i couldn't find any college"]
response i could n't find any college, bs {'control': 'public', 'region': 'southwest', 'area': 'computer science', 'n

In [ ]:
with open("predictions.json","r",encoding="utf8") as infile:
    predictions = json.load(infile)

Belief-State Analysis

In [26]:
# check accuracy of belief state (prediction)
def check_bs(gold_bs:str,predicted_bs:str,want_print=False):
    # we want to check true positives (gold+pred), false positive (only pred) and false negatives (only gold) for predicting slots and/or values
    slots_TP, slots_FP, slots_FN = 0, 0, 0
    values_TP, values_FP, values_FN = 0, 0, 0
    sv_TP, sv_FP, sv_FN = 0, 0, 0

    # transform gold belief state into dictionary
    gold = dict()
    gold_bs = gold_bs.replace("belief : ","")
    gold_list = gold_bs.split(";")
    for slot_value in gold_list:
        if "=" in slot_value:
            splitted = slot_value.split("=")
            slot = splitted[0].strip()
            value = splitted[1].strip()
            if slot not in gold:
                gold[slot] = [value]
            else:
                gold[slot] += [value]
    if want_print:
        print("gold: ",gold)

    # transform predicted belief state into dictionary
    predicted = dict()
    predicted_bs = predicted_bs.replace("belief : ","")
    predicted_list = predicted_bs.split(";")
    for slot_value in predicted_list:
        if "=" in slot_value:
            splitted = slot_value.split("=")
            slot = splitted[0].strip()
            value = splitted[1].strip()
            if slot not in predicted:
                predicted[slot] = [value]
            else:
                predicted[slot] += [value]
    if want_print:
        print("predicted: ",predicted)

    # consider counts for slots -> don't care about how many values were recognized / if they are correct; only recognition of slots matter
    gold_slots = list(gold.keys())
    predicted_slots = list(predicted.keys())
    slots_FP = len([item for item in predicted_slots if item not in gold_slots])
    slots_FN = len([item for item in gold_slots if item not in predicted_slots])
    slots_TP = len([item for item in predicted_slots if item in gold_slots]) 
    if want_print:
        print("slots:\tFP: ",slots_FP,"\tFN: ",slots_FN,"\tTP: ",slots_TP)

    # consider counts for values -> don't care if the values are matched properly to the slots; only recognition of values matter
    gold_values = [val for vals in gold.values() for val in vals]
    predicted_values = [val for vals in predicted.values() for val in vals]
    values_FP = len([item for item in predicted_values if item not in gold_values])
    values_FN = len([item for item in gold_values if item not in predicted_values])
    values_TP = len([item for item in predicted_values if item in gold_values]) 
    if want_print:
        print("values:\tFP: ",values_FP,"\tFN: ",values_FN,"\tTP: ",values_TP)

    # consider counts for slot-value-tuples -> slot as well as value need to match!
    gold_tuples = [(s,v) for s,vals in gold.items() for v in vals]
    predicted_tuples = [(s,v) for s,vals in predicted.items() for v in vals]
    sv_FP = len([item for item in predicted_tuples if item not in gold_tuples])
    sv_FN = len([item for item in gold_tuples if item not in predicted_tuples])
    sv_TP = len([item for item in predicted_tuples if item in gold_tuples]) 
    if want_print:
        print("slot-value pairs:\tFP: ",sv_FP,"\tFN: ",sv_FN,"\tTP: ",sv_TP)

    return {"slots":(slots_FP,slots_FN,slots_TP),"values":(values_FP,values_FN,values_TP),"slots_values":(sv_FP,sv_FN,sv_TP)}

In [44]:
# run some examples
check_bs("belief : area = architecture ; state = california"," belief : state = california",want_print=True)
print()
check_bs("belief : area = architecture"," belief : state = california",want_print=True)
print()
check_bs("belief : state = california"," belief : area = architecture ;  state = california",want_print=True)
print()
check_bs("belief : area = architecture ; area = engineering"," belief : area = architecture",want_print=True)
print()
check_bs("belief : area = architecture ; area = engineering"," belief : area = architecture ; area = engineering",want_print=True)
print()
check_bs("belief : area = architecture ; state = california"," belief : state = california",want_print=True)

gold:  {'area': ['architecture'], 'state': ['california']}
predicted:  {'state': ['california']}
slots:	FP:  0 	FN:  1 	TP:  1
values:	FP:  0 	FN:  1 	TP:  1
slot-value pairs:	FP:  0 	FN:  1 	TP:  1

gold:  {'area': ['architecture']}
predicted:  {'state': ['california']}
slots:	FP:  1 	FN:  1 	TP:  0
values:	FP:  1 	FN:  1 	TP:  0
slot-value pairs:	FP:  1 	FN:  1 	TP:  0

gold:  {'state': ['california']}
predicted:  {'area': ['architecture'], 'state': ['california']}
slots:	FP:  1 	FN:  0 	TP:  1
values:	FP:  1 	FN:  0 	TP:  1
slot-value pairs:	FP:  1 	FN:  0 	TP:  1

gold:  {'area': ['architecture', 'engineering']}
predicted:  {'area': ['architecture']}
slots:	FP:  0 	FN:  0 	TP:  1
values:	FP:  0 	FN:  1 	TP:  1
slot-value pairs:	FP:  0 	FN:  1 	TP:  1

gold:  {'area': ['architecture', 'engineering']}
predicted:  {'area': ['architecture', 'engineering']}
slots:	FP:  0 	FN:  0 	TP:  1
values:	FP:  0 	FN:  0 	TP:  2
slot-value pairs:	FP:  0 	FN:  0 	TP:  2

gold:  {'area': ['architectu

{'slots': (0, 1, 1), 'values': (0, 1, 1), 'slots_values': (0, 1, 1)}

In [27]:
def get_prec_rec_f(fp,fn,tp,want_print=False):
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = 2 * ((prec * rec) / (prec + rec))
    if want_print:
        print("prec: ",prec,"\trec: ",rec,"\tf1: ",f1)

In [28]:
# iterate over test data, make soloist predictions and get precision, recall and f1-score
slots = {"FP":0,"FN":0,"TP":0}
values = {"FP":0,"FN":0,"TP":0}
slots_values = {"FP":0,"FN":0,"TP":0}
for test_dialogue in predictions:
    gold_bs = test_dialogue["belief"]
    gold_bs = str("belief : "+gold_bs)
    predicted_bs = test_dialogue["predicted_belief"]
    predicted_bs_temp = "belief : "
    for s,v in predicted_bs.items():
        if len(predicted_bs_temp) == 9:
            predicted_bs_temp += str(s + " = " + v)
        else:
            predicted_bs_temp += str(" ; " + s + " = " + v)
    predicted_bs = predicted_bs_temp
    counts = check_bs(gold_bs,predicted_bs)
    slots["FP"] += counts["slots"][0]
    slots["FN"] += counts["slots"][1]
    slots["TP"] += counts["slots"][2]
    values["FP"] += counts["values"][0]
    values["FN"] += counts["values"][1]
    values["TP"] += counts["values"][2]
    slots_values["FP"] += counts["slots_values"][0]
    slots_values["FN"] += counts["slots_values"][1]
    slots_values["TP"] += counts["slots_values"][2]
print("slots: ")
print("\t",slots)
print("\t",get_prec_rec_f(slots["FP"],slots["FN"],slots["TP"],True))
print("values: ")
print("\t",values)
print("\t",get_prec_rec_f(values["FP"],values["FN"],values["TP"],True))
print("slot values: ")
print("\t",slots_values)
print("\t",get_prec_rec_f(slots_values["FP"],slots_values["FN"],slots_values["TP"],True))


slots: 
	 {'FP': 169, 'FN': 102, 'TP': 136}
prec:  0.4459016393442623 	rec:  0.5714285714285714 	f1:  0.5009208103130754
	 None
values: 
	 {'FP': 146, 'FN': 89, 'TP': 159}
prec:  0.521311475409836 	rec:  0.6411290322580645 	f1:  0.5750452079566003
	 None
slot values: 
	 {'FP': 180, 'FN': 123, 'TP': 125}
prec:  0.4098360655737705 	rec:  0.5040322580645161 	f1:  0.45207956600361665
	 None


In [ ]:
# note: don't forget to include some qualitative analyses -> manually look at some beliefstate predictions

Analysis of system output

In [29]:
bleu_scores = [] # collegt bleu scores for all test-dialogues

In [32]:
# example
gold_out_ex = "system : there's many. is there anything else you want from the college?"
gold_out_ex = gold_out_ex.replace("system : ","")
predicted_out_ex = "alright . there is a lot of colleges left . what specifically are you looking for ?"
bleu_ex = corpus_bleu([gold_out_ex.split()],[predicted_out_ex.split()])
print(bleu_ex)

1.1808001696991507e-231


/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram o

In [33]:
golds_out = []
preds_out = []
for test_dialogue in predictions:
    gold_out = test_dialogue["reply"]
    predicted_out = test_dialogue["predicted_reply"]
    if "system" in gold_out:
        gold_out = gold_out.replace("system : ","")
        if len("system") > 1: # we actually have some gold-system-reply
            golds_out += [gold_out.split()]
            preds_out += [predicted_out.split()]
bleu_score = corpus_bleu(golds_out,preds_out)
print(bleu_score)

1.3214619082097209e-155


/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mount/studenten-temp1/users/godberja/SpokenDialogSystems/soloist/soloist_env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates)
print(score) # returns 1 because the candidate is inside the references

In [ ]:
# note: don't forget to include some qualitative analyses -> manually look at some text generations